<a href="https://colab.research.google.com/github/fberetta8/ML_in_Finance_Spring_2025_ETH/blob/main/Notebook1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The role of volatility in BS formula
This exercise should serve as a gentle introduction to the Black-Scholes formula, which plays a major role in quantitative finance. In this exercise we aim to write a NN which approximates the dependence of the price of an European call option on the volatility of the model.

In [ ]:
# Libraries
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import scipy.stats as si
import matplotlib.pyplot as plt

## Black-Scholes Formula

We consider the Black-Scholes formula for the price of a European call option:

$$
\begin{aligned}             
C(X,r,\sigma,T,K) &= X \Phi(d_1) - K e^{-rT} \Phi(d_2), \text{ where }\\
d_1 &=\frac{\ln(X/K) + (r + \frac{1}{2} \sigma^2)T}{\sigma \sqrt{T}}\\
d_2 &= d_1 - \sigma \sqrt{T}.
\end{aligned}
$$

In the above:
- $X$ is the price of the underlying asset,
- $r$ is the risk-free interest rate,
- $\sigma$ is the volatility of the market,
- $T$ is the time to maturity,
- $K$ is the strike price,
- $\Phi$ is the cumulative distribution function (CDF) of the standard normal distribution.

We shall discuss this model in detail in another lecture. For now, we fix the values:

$$
X=2, \quad r=0, \quad T=1, \quad K=1 \quad \sigma \in [0.01,1].
$$

Substituting these into the formula:

$$
\begin{aligned}             
C(\sigma) &= 2\Phi(d_1) - \Phi(d_2),\\
d_1 &= \frac{\ln(2) + \frac{1}{2} \sigma^2}{\sigma}\\
d_2 &= \frac{\ln(2) - \frac{1}{2} \sigma^2}{\sigma}.
\end{aligned}
$$


In [ ]:
# Black-Scholes formula
def black_scholes_formula(sigma, X=0.5, K=1, T=1, r=0):

    # TODO: Implement the BS formula described above.

In [ ]:
# Generate Training Data
def generate_data(num_samples):

    # TODO: Sample `sigma` values from a uniform distribution (0.01 to 1)

    # TODO: Compute option prices using `black_scholes_formula`

    # TODO: Convert to PyTorch tensors

    return X_train, y_train

In [ ]:
# Test your function
X_train, y_train = generate_data(10)
print("Sample Data:", X_train[:5], y_train[:5])

In [ ]:
# Define Neural Network Model
class BlackScholesNN(nn.Module):
    def __init__(self, hidden_size=32, num_layers=3):
        super(BlackScholesNN, self).__init__()

        # TODO: Implement a feedforward NN with `num_layers` hidden layers

    def forward(self, x):
        return self.model(x)

In [ ]:
# Test your Model
model = BlackScholesNN(hidden_size=32, num_layers=3)
print(model)

In [ ]:
# Train the model
def train_model(num_samples, num_layers=3, epochs=500, learning_rate=0.01):
    X_train, y_train = generate_data(num_samples)

    model = BlackScholesNN(num_layers=num_layers)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(epochs):
        optimizer.zero_grad()
        predictions = model(X_train)
        loss = criterion(predictions, y_train)
        loss.backward()
        optimizer.step()

        if epoch % 50 == 0:
            print(f"Epoch {epoch}: Loss = {loss.item():.6f}")

    return model, X_train, y_train

**Test 1**: Fix the network structure and vary the **number of training samples**.



In [ ]:
model_5, X_train_5, y_train_5 = train_model(num_samples=5)
model_10, X_train_10, y_train_10 = train_model(num_samples=10)
model_100, X_train_100, y_train_100 = train_model(num_samples=100)

# Generate Test Data
sigma_test = np.linspace(0.01, 1, 100).reshape(-1, 1)
sigma_test_tensor = torch.tensor(sigma_test, dtype=torch.float32)
true_prices = black_scholes_formula(sigma_test)

# Predict Prices Using Trained Models
with torch.no_grad():
    pred_5 = model_5(sigma_test_tensor).numpy()
    pred_10 = model_10(sigma_test_tensor).numpy()
    pred_100 = model_100(sigma_test_tensor).numpy()

# Plot Results
plt.figure(figsize=(10, 6))
plt.plot(sigma_test, true_prices, label="True Black-Scholes", linewidth=3, color="black")
plt.plot(sigma_test, pred_5, label="Train Size = 5", linestyle="dashed", color="red")
plt.plot(sigma_test, pred_10, label="Train Size = 10", linestyle="dashed", color="blue")
plt.plot(sigma_test, pred_100, label="Train Size = 100", linestyle="dashed", color="green")

plt.xlabel("Volatility (σ)")
plt.ylabel("Option Price")
plt.title("Black-Scholes Approximation with Different Training Set Sizes")
plt.legend()
plt.show()

**Test 2**: Fix the train set and vary the **number of hidden layers**.


In [ ]:
model_1, _, _ = train_model(num_samples=100, num_layers=1)
model_3, _, _ = train_model(num_samples=100, num_layers=3)
model_5, _, _ = train_model(num_samples=100, num_layers=5)

# Generate Test Data
sigma_test = np.linspace(0.01, 1, 100).reshape(-1, 1)
sigma_test_tensor = torch.tensor(sigma_test, dtype=torch.float32)
true_prices = black_scholes_formula(sigma_test)

# Predict Prices Using Trained Models
with torch.no_grad():
    pred_1 = model_1(sigma_test_tensor).numpy()
    pred_3 = model_3(sigma_test_tensor).numpy()
    pred_5 = model_5(sigma_test_tensor).numpy()

# Plot Results
plt.figure(figsize=(10, 6))
plt.plot(sigma_test, true_prices, label="True Black-Scholes", linewidth=3, color="black")
plt.plot(sigma_test, pred_1, label="1 Hidden Layer", linestyle="dashed", color="red")
plt.plot(sigma_test, pred_3, label="3 Hidden Layers", linestyle="dashed", color="blue")
plt.plot(sigma_test, pred_5, label="5 Hidden Layers", linestyle="dashed", color="green")

plt.xlabel("Volatility (σ)")
plt.ylabel("Option Price")
plt.title("Black-Scholes Approximation with Different Network Architectures")
plt.legend()
plt.show()